In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def encode_categorical(df, cols):
    
    for col in cols:
        # Leave NaN as it is.
        le = LabelEncoder()
        #not_null = df[col][df[col].notnull()]
        df[col] = df[col].fillna('nan')
        df[col] = pd.Series(le.fit_transform(df[col]), index=df.index)

    return df


In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns毎に処理
        col_type = df[col].dtypes
        if col_type in numerics: #numericsのデータ型の範囲内のときに処理を実行. データの最大最小値を元にデータ型を効率的なものに変更
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [ ]:
store_list = ["CA_1", "CA_2", "CA_3", "CA_4", "TX_1", "TX_2", "TX_3", "WI_1", "WI_2", "WI_3"]
datas = pd.DataFrame()
for i, store in enumerate(store_list):
    buff = pd.read_pickle(f"/kaggle/input/m5mylib/{store}_interpolate.pickle")
    
    #並び順を戻す
    store = pd.read_csv(f"/kaggle/input/m5mylib/add_Clean_Analysis_{store}.csv").pipe(reduce_mem_usage)
    store = encode_categorical(
       store, ["item_id"]
    ).pipe(reduce_mem_usage)
    gc.collect()
    
    store = store[["item_id", "week", "wday", "variable"]]
    store["variable"] = store["variable"].apply(lambda x: x[-2:])
    store["variable"] = store["variable"].astype(np.int8)
    store = store.loc[store["variable"] >= 13]
    
    # item_idの順番を整える
    buff = pd.merge(store, buff)
    
    buff["store_id"] = i
    
    # 線形補間でNaNデータを埋める
    buff["sell_num"] = buff.groupby('item_id')["value"].apply(lambda group: group.interpolate())
    
    # 全体の固有ID値を振る
    ids = buff[["item_id"]]
    ids = ids[~ids.duplicated()]
    values = (3049*i) + np.arange(3049)
    dictionary = {k: v for k, v in zip(np.array(ids["item_id"]), values)}
    buff["id"] = buff["item_id"].map(dictionary)
    
    datas = pd.concat([datas, buff])
    #datas[datas["store_id"] == i].sort_values(["item_id","date"]).to_pickle(f"{store}.pickle")
    #product[product["store_id"] == i].to_pickle(f"product_{store}.pickle")
    print(f"{str(i+1)}/9")

In [ ]:
del buff, store
gc.collect()

In [ ]:
datas = datas[datas["variable"] >= 13]

In [ ]:
datas["store_id"] = datas["store_id"].astype('int8')
datas["id"] = datas["id"].astype("int16")

In [ ]:
calender = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
calender = calender[calender["year"] >= 2013]

In [ ]:
calender.drop(["weekday", "year", "wday", "month", "d", "event_type_1", "event_type_2"], axis=1, inplace=True)

In [ ]:
calender = encode_categorical(
    calender,["event_name_1", "event_name_2", "snap_CA", "snap_TX", "snap_WI"]
).pipe(reduce_mem_usage)

In [ ]:
datas = pd.merge(datas, calender, how='left',on=["date"])

In [ ]:
datas.dtypes

In [ ]:
#priceも扱う
price = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv").pipe(reduce_mem_usage)
price = encode_categorical(
   price, ["store_id", "item_id"],
).pipe(reduce_mem_usage)

In [ ]:
calender.drop(["event_name_1", "event_name_2", "snap_CA", "snap_TX", "snap_WI"], axis=1, inplace=True)

In [ ]:
calender = pd.merge(calender,price)

In [ ]:
calender.drop("wm_yr_wk", axis=1, inplace=True)

In [ ]:
calender

In [ ]:
datas

In [ ]:
datas['date'] = pd.to_datetime(datas['date'])
calender['date'] = pd.to_datetime(calender['date'])

In [ ]:
del price
gc.collect()

In [ ]:
datas = pd.merge(datas, calender, on=['store_id', 'item_id', 'date'], how='left')

In [ ]:
del calender
gc.collect()

In [ ]:
#1日前の売り上げ個数を特徴に追加
#datas["prev_1"] = datas.groupby("id")["sell_num"].transform(lambda x:x.shift(1))

#2日前の売り上げ個数を特徴に追加
#datas["prev_2"] = datas.groupby("id")["sell_num"].transform(lambda x:x.shift(2))

#7日前の売り上げ個数を特徴に追加
#datas["prev_7"] = datas.groupby("id")["sell_num"].transform(lambda x:x.shift(7))

#8日前
#datas["prev_8"] = datas.groupby("id")["sell_num"].transform(lambda x:x.shift(8))

#9日前
#datas["prev_9"] = datas.groupby("id")["sell_num"].transform(lambda x:x.shift(9))

#14日前の売り上げ個数を特徴に追加
#datas["prev_14"] = datas.groupby("id")["sell_num"].transform(lambda x:x.shift(14))

#############

#28日前の売り上げ個数を特徴に追加
datas["prev_28"] = datas.groupby("id")["sell_num"].transform(lambda x:x.shift(28))

#29日前の売り上げ個数を特徴に追加
datas["prev_29"] = datas.groupby("id")["sell_num"].transform(lambda x:x.shift(29))

#30日前の売り上げ個数を特徴に追加
datas["prev_30"] = datas.groupby("id")["sell_num"].transform(lambda x:x.shift(30))

#35日前の売り上げ個数を特徴に追加
#datas["prev_35"] = datas.groupby("id")["sell_num"].transform(lambda x:x.shift(35))

#半年前(182日前)の売り上げ個数を特徴に追加
#datas["prev_182"] = datas.groupby("id")["sell_num"].transform(lambda x:x.shift(182))

#1年前(365日前)の売り上げ個数を特徴に追加
#datas["prev_365"] = datas.groupby("id")["sell_num"].transform(lambda x:x.shift(365))

In [ ]:
#1日前の価格を特徴に追加
datas["p_prev_1"] = datas.groupby("id")["sell_price"].transform(lambda x:x.shift(1))

#2日前の価格を特徴に追加
#datas["p_prev_2"] = datas.groupby("id")["sell_price"].transform(lambda x:x.shift(2))

#3日前の価格を特徴に追加
#datas["p_prev_3"] = datas.groupby("id")["sell_price"].transform(lambda x:x.shift(3))

#7日前の価格を特徴に追加
#datas["p_prev_7"] = datas.groupby("id")["sell_price"].transform(lambda x:x.shift(7))

#14日前の価格を特徴に追加
#datas["p_prev_14"] = datas.groupby("id")["sell_price"].transform(lambda x:x.shift(14))

#28日前の価格を特徴に追加
#datas["p_prev_28"] = datas.groupby("id")["sell_price"].transform(lambda x:x.shift(28))
#あまりラグを大きくしすぎるとアイテムによってはデータが全然ない状況に...

In [ ]:
#for size in [7, 30]:
#        datas[f"rolling_mean_t{size}"] = datas.groupby(["id"])["sell_num"].transform(
#            lambda x: x.shift(7).rolling(size).mean()
#        )
for size in [7, 14, 28]:
        datas[f"rolling_mean_t{size}"] = datas.groupby(["id"])["sell_num"].transform(
            lambda x: x.shift(28).rolling(size).mean()
        )
#for size in [7, 30, 60, 90, 180]:
#        datas[f"rolling_mean_t{size}"] = datas.groupby(["id"])["sell_num"].transform(
#            lambda x: x.shift(28).rolling(size).mean()
#        )

In [ ]:
#for size in [7, 30]:
#        datas[f"rolling_std_t{size}"] = datas.groupby(["id"])["sell_num"].transform(
#            lambda x: x.shift(7).rolling(size).std()
#        )
for size in [7, 14, 28]:
        datas[f"rolling_std_t{size}"] = datas.groupby(["id"])["sell_num"].transform(
            lambda x: x.shift(28).rolling(size).std()
        )

In [ ]:
datas["price_change_t1"] = (datas["p_prev_1"] - datas["sell_price"]) / (
        datas["p_prev_1"]
    )

In [ ]:
datas["rolling_price_std_t7"] = datas.groupby(["id"])["sell_price"].transform(
        lambda x: x.rolling(7).std()
    )
datas["rolling_price_std_t28"] = datas.groupby(["id"])["sell_price"].transform(
        lambda x: x.rolling(28).std()
    )

In [ ]:
 attrs = [
        "year",
        "quarter",
        "month",
        "week",
        "day",
        "dayofweek",
        #"is_year_end",
        #"is_year_start",
        #"is_quarter_end",
        #"is_quarter_start",
        #"is_month_end",
        #"is_month_start",
    ]

for attr in attrs:
    dtype = np.int16 if attr == "year" else np.int8
    datas[attr] = getattr(datas["date"].dt, attr).astype(dtype)

datas["is_weekend"] = datas["dayofweek"].isin([5, 6]).astype(np.int8)

In [ ]:
datas

In [ ]:
datas.to_pickle("/kaggle/working/merge_data2.pickle")